In [1]:
import os

os.environ["SEED"] = "0"

import cocoex
import matplotlib.pyplot as plt
import nevergrad as ng
import numpy as np
import scipy

from src.solver.BBOB_POP_Solver import BBOB_POP_Solver
from src.instance.BBOB_Instance import BBOB_Instance

In [2]:
suite = cocoex.Suite("bbob", "", "function_indices:1 dimensions:2 instance_indices:1-15")
# 6 dimensions X 24 functions X 15 instances = 2160 problems
len(list(suite))

15

In [3]:
instance = BBOB_Instance(
    function_index=1,
    dimension=2,
    instance_index=1,
    cut_off_cost=50.0,
    cut_off_time=5.0,
)
instance

BBOB_Instance(problem_id=bbob_f001_i01_d02)

In [6]:
solver = BBOB_POP_Solver()

result = solver._solve(
    prefix="test",
    solver=solver,
    instance=instance,
    features_time=0
)
result.cost, result.time, result.cut_off_cost, result.cut_off_time

(0.6653308868408203, 0.6653308868408203, 50.0, 5.0)

In [7]:
solver.solve(
    instance=instance,
    prefix="test",
    calculate_features=False,
    cache=False
)

[2025-07-11 07:00:28] DEBUG     solve(prefix=test, solver=Solver(id=2253826169615980878), instance=BBOB_Instance(problem_id=bbob_f001_i01_d02))
[2025-07-11 07:00:29] DEBUG     Solver.Result(prefix=test, solver=Solver(id=2253826169615980878), instance=BBOB_Instance(problem_id=bbob_f001_i01_d02), cost=0.86, time=0.86, cut_off_cost=50.00, cut_off_time=5.00, cached=False, surrogate=False, error=False)


<Future at 0x1910342ac80 state=finished returned Result>

In [ ]:
problem = suite[0]

pso_config = ng.families.ConfPSO(
    popsize=20,           # Population size
    omega=0.729,          # Inertia weight
    phip=1.49618,         # Cognitive parameter
    phig=1.49618,         # Social parameter
)

optimizer = pso_config(2, budget=100000)

recommendation = optimizer.minimize(problem, max_time=100)
recommendation.value, problem.evaluations, problem.final_target_hit

In [ ]:
problem = suite[0]

de_config = ng.families.DifferentialEvolution(
    popsize=20,
    crossover=0.8,
)

optimizer = de_config(2, budget=10000)

recommendation = optimizer.minimize(problem, max_time=1)
recommendation.value, problem.evaluations, problem.final_target_hit